### Homework: Vector Search

In this homework, we will learn more about vector search and embedding. Like in the module, we will use Qdrant and fastembed.

Remember to start docker first:

```
docker run -p 6333:6333 -p 6334:6334 \
   -v "$(pwd)/qdrant_storage:/qdrant/storage:z" \
   qdrant/qdrant
```

### Embeddings

Qdrant uses [fastembed](https://github.com/qdrant/fastembed) under the hood to turn text into vectors.

`pip install fastembed`

`pip install -q "qdrant-client[fastembed]>=1.14.2"`

In [18]:
from fastembed import TextEmbedding
import numpy as np
from numpy.linalg import norm
import requests 
from qdrant_client import QdrantClient

### Q1. Embedding the query

Embed the query: `'I just discovered the course. Can I join now?'`.  Use the `'jinaai/jina-embeddings-v2-small-en'` model.

You should get a numpy array of size 512.

What's the minimal value in this array?

* -0.51
* -0.11
* 0
* 0.51

In [2]:
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"
query = "I just discovered the course. Can I join now?"
collection_name = "zoomcamp-rag"

We previously create a collection in semantic_search.ipynb:

```
# Define the collection name
collection_name = "zoomcamp-rag"

# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)
```


In [3]:
model = TextEmbedding(model_name=model_handle)
embedding = list(model.embed([query]))[0] # get the first embedding
query_embedding = np.array(embedding) # convert to an array
print(np.min(query_embedding))

-0.11726373885183883


### A1. The minimum value in the embedded query array is -0.11.

### Cosine Similarity

The vectors that our embedding model returns are already normailized: their length is 1.0.

You can check that by using the `norm` function:

`np.linalg.norm(q)`

Which means that we can simply compute the dot product between the two vectors to learn the cosine similarity between them.

For example, if you compute the cosine of the query vector with itself, the result will be 1.0:

`q.dot(q)`

In [4]:
norm(query_embedding)

np.float64(1.0)

In [5]:
query_embedding.dot(query_embedding)

np.float64(1.0000000000000002)

### Q2. Cosine similarity with another vector

Now let's embed this document:

`doc = 'Can I still join the course after the start date?'`

What's the cosine similarity between the vector for the query and the vector for the document?

* 0.3
* 0.5
* 0.7
* 0.9

In [6]:
doc = 'Can I still join the course after the start date?'
doc_embedding = list(model.embed([doc]))[0]
doc_embedding = np.array(doc_embedding) # convert to an array

In [7]:
query_embedding.dot(doc_embedding)

np.float64(0.9008528895674548)

### A2. The cosine similarity between the vector for the query and the vector for the document is 0.9.

### Q3. Ranking by cosine

For Q3 and Q4 we will use these documents:


In [8]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

Compute the embeddings for the text field, and compute the cosine between the query vector and all the documents.

What's the document index with the highest similarity? (Indexing starts from 0):

* 0
* 1
* 2
* 3
* 4

Hint: If you put al the embeddings of the text field in one matrix V (a single 2-dimensional numpy array), then computing the cosine becomes matrix multiplication:

`V.dot(q)`

Create the embeddings for the text field of the documents:

In [9]:
doc_embeddings = [np.array(list(model.embed([doc["text"]])))[0] for doc in documents]

We already have our `query_embedding` from above:

Calculate the cosine similarity

In [10]:
cosine_similarities = [np.dot(query_embedding, doc_emb) / (norm(query_embedding) * norm(doc_emb))
    for doc_emb in doc_embeddings
]
cosine_similarities

[np.float64(0.7629684696540238),
 np.float64(0.8182378150042889),
 np.float64(0.8085397398734189),
 np.float64(0.7133079015686243),
 np.float64(0.7304499234333598)]

Find the most similar document

In [11]:
max_index = np.argmax(cosine_similarities)
print(f"Highest similarity score: {cosine_similarities[max_index]:.4f} at index {max_index}")

Highest similarity score: 0.8182 at index 1


### A3. The most similar document has index = 1.  

### Q4. Ranking by cosine, version two

Now, let's calculate a new field, which is a concatenation of `question` and `text`:

`full_text = doc['question'] + ' ' + doc['text']`

Embed this field and compute the cosine between it and the query vector.  What's the highest scoring document?

* 0
* 1
* 2
* 3
* 4

Is it different from Q3?  If yes, why?

In [12]:
documents = [
    {
        'full_text': f"{doc['question']} {doc['text']}", 
        **doc
    } 
    for doc in documents
]

In [13]:
doc_embeddings = np.array(list(model.embed([doc['full_text'] for doc in documents])))


In [14]:
cosine_similarities = [
    np.dot(query_embedding, doc_emb) / (norm(query_embedding) * norm(doc_emb))
    for doc_emb in doc_embeddings
]

In [15]:
max_index = np.argmax(cosine_similarities)
print(f"Highest similarity score: {cosine_similarities[max_index]:.4f} at index {max_index}")

Highest similarity score: 0.8515 at index 0


### A4. The highest similarity score is now index 0. By combining the question and text, we create richer context vectors, since it captures both the intent (the question) and the contect (the text).

### Q5. Selecting the embedding model

Now let's select a smaller embedding model.  What's the smallest dimensionality for models in fastembed?

* 128
* 256
* 384
* 512

One of these models is [`BAAI/bge-small-en`](https://huggingface.co/BAAI/bge-small-en-v1.5). Let's use it.

### A5. According to the model card at Huggingface, the dimension of the `BAAI/bge-small-en` is 384.

### Q6. Indexing with qdrant

For the last question, we will use more documents.

We will select only FAQ records from our ml zoomcamp.

In [ ]:
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [19]:
documents[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp'}

Add them to qdrant using the model form in Q5.

When adding the data, use both question and answer fields:

`text = doc['question'] + ' ' + doc['text']`

In [ ]:
# Initialize the client
#client = QdrantClient("localhost", port=6333) # for production
# client.set_model("BAAI/bge-small-en")
client = QdrantClient(":memory:", model_name="BAAI/bge-small-en") # for small experiments

In [22]:
combined_texts = [f"{doc['question']} {doc['text']}" for doc in documents]
metadata = [
    {
        "section": doc['section'],
        "course": doc['course'],
        "original_text": doc['text'],
        "original_question": doc['question']
    } for doc in documents
]

In [23]:
document_ids = client.add(
    collection_name="hmwk2_ml_zoomcamp_qa",
    documents=combined_texts,
    metadata=metadata,
    batch_size=32 
)


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

After the data is inserted, use the question from Q1 for querying the collection.

What's the highest score in the results? (the score for the first returned record)):

* 0.97
* 0.87
* 0.77
* 0.67

In [24]:
print(query)

I just discovered the course. Can I join now?


In [25]:
results = client.query(
    collection_name="hmwk2_ml_zoomcamp_qa",
    query_text=query,
    limit=1
)
print(results)

[QueryResponse(id='a458073abd3a43698870fbb76d4dba47', embedding=None, sparse_embedding=None, metadata={'document': 'The course has already started. Can I still join it? Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'course': 'machine-learning-zoomcamp', 'original_text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligi

In [26]:
for hit in results:
    print(hit.score)

0.8703173344744922


### A6. The highest score in the returned results was 0.87. 



